In [16]:
# 첫 번째 셀 - 설정
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    FewShotChatMessagePromptTemplate,
)
from langchain.memory import ChatMessageHistory
from langchain_openai import ChatOpenAI

# 채팅 기록을 저장할 메모리 생성
history = ChatMessageHistory()

# 챗봇 설정
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# Few-shot 예제 설정
examples = [
    {"human": "The title of the movie is 'Top Gun'", "ai": "🛩️👨‍✈️🔥"},
    {"human": "The title of the movie is 'The Godfather'", "ai": "👨‍👨‍👦🔫🍝"},
    {"human": "The title of the movie is 'Jaws'", "ai": "🦈🏊‍♂️😱"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{human}"),
        ("ai", "{ai}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="영화 제목을 받으면 딱 3개의 이모지로만 응답하세요."),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

# 두 번째 셀 - 영화 질문하기
# 첫 번째 영화
first_message = HumanMessage(content="The title of the movie is Matrix")
first_response = chain.invoke({"messages": [first_message]})
history.add_message(first_message)
history.add_message(first_response)
print("첫 번째 영화:", first_response.content)

# 두 번째 영화
second_message = HumanMessage(content="The title of the movie is Titanic")
second_response = chain.invoke({"messages": [second_message]})
history.add_message(second_message)
history.add_message(second_response)
print("두 번째 영화:", second_response.content)

# 세 번째 셀 - 메모리에서 첫 번째 영화와 이모지 확인하기
messages = history.messages
first_movie = messages[0].content.split("is ")[-1].strip()  # "Matrix" 부분만 추출
first_emoji = messages[1].content
print(f"첫 번째로 물어본 영화: {first_movie} {first_emoji}")

첫 번째 영화: 💊🕶️💻
두 번째 영화: 🚢💔🌊
첫 번째로 물어본 영화: Matrix 💊🕶️💻
